In [1]:
from preprocessing_utils import load_all_files, preprocess_documents
from pathlib import Path
import os
from api import GOOGLE_API_KEY
import google.generativeai as genai
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
genai.configure(api_key=GOOGLE_API_KEY)

c:\Users\mikel\Documents\GitHub\multimodal-rag\.venv\Lib\site-packages\deepeval\__init__.py:54: UserWarning: You are using deepeval version 2.6.6, however version 2.6.7 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(


## Text only

In [ ]:
papers_path = Path("papers/selection")
docs = load_all_files(papers_path)
docs = preprocess_documents(docs) 

In [2]:
from langchain_core.prompts import PromptTemplate

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use around three sentences for your answer and keep it concise.
Do not ever mention that you are using the context as source.

Question: {question}

Context:
{context}

Answer:"    
"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [3]:
# from langchain import hub
# from langchain_chroma import Chroma
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from qdrant_client import models
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs)
qdrant_client = QdrantClient(path="papers_db")
# client.create_collection(
#     collection_name="selection",
#     vectors_config=models.VectorParams(
#         size=768,
#         distance=models.Distance.COSINE
#     )

# )
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
# vectorstore = Chroma.from_documents(
#     documents=splits,
#     embedding=embeddings,
#     persist_directory="selection_db"
#  )
# vectorstore = Chroma(
#     persist_directory="selection_db",
#     embedding_function=embeddings
# )
vectorstore = QdrantVectorStore(
    client=qdrant_client,
    collection_name="selection",
    embedding=embeddings,
    distance=models.Distance.COSINE
)
# vectorstore.add_documents(splits)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
prompt = custom_rag_prompt


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

# rag_chain.invoke("What is Task Decomposition?")
system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer the question."
    "If you don't know the answer, just say that you don't know."
    "Keep your answer concise and to the point."
    "Use LaTeX for any math equation."
    # "Use around three sentences for your answer and keep it concise."
    "Do not ever mention that you are using the context as source."
    "Here is context for your answer:"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# rag_chain.invoke({"input": "What kind of material is MAPbI3?"})

In [28]:
response = rag_chain.invoke({"input":
"Give me an example architecture of a graph neural network used to predict physical properties of crystals and describe the internal architecture."
                             })
print(response["context"])
print(response["answer"])

[Document(metadata={'producer': 'iText® 5.3.5 ©2000-2012 1T3XT BVBA (SPRINGER SBM; licensed version)', 'creator': 'Springer', 'creationdate': '2021-08-31T00:12:39+05:30', 'keywords': '', 'crossmarkdomains[1]': 'springer.com', 'moddate': '2021-08-31T07:48:08+02:00', 'subject': 'Communications Materials, doi:10.1038/s43246-021-00194-3', 'doi': '10.1038/s43246-021-00194-3', 'author': 'Jiucheng Cheng', 'crossmarkdomains[2]': 'springerlink.com', 'title': 'A geometric-information-enhanced crystal graph network for predicting properties of materials', 'source': 'papers\\selection\\Cheng et al. - 2021 - A geometric-information-enhanced crystal graph net.pdf', 'total_pages': 11, 'page': 1, 'page_label': '2', '_id': '64a0bfbb3d48419a91198657aff450eb', '_collection_name': 'selection'}, page_content='aggregation process. In material prediction domain, the geometrical Crystal graph deﬁ nition and the introduction of geometric\nstructure information like spatial distance and direction is alsoinforma

## Multimodal

In [6]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")

In [7]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, SystemMessage
import base64

# Function to encode an image as base64
def encode_image(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode("utf-8")

# Encode the image(s)
image_path = "image.jpg"  # Replace with your image path
image_base64 = encode_image(image_path)

# Define system message (instructions for the AI)
system_template = "You are an AI assistant that analyzes images and answers user queries."
system_message = SystemMessagePromptTemplate.from_template(system_template)

# Define human message (text + image context)
human_template = "{text}"  # Placeholder for user input text
human_message = HumanMessagePromptTemplate.from_template(human_template)

# Create the chat prompt template
chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

# Format the chat prompt with user input
formatted_prompt = chat_prompt.format_messages(
    text="What can you tell me about this image?",
)

# Append image to the human message dynamically
formatted_prompt.append(
    HumanMessage(
        content=[
            {"type": "text", "text": "Here is the image context:"},
            {"type": "image_url", "image_url": f"data:image/jpeg;base64,{image_base64}"}
        ]
    )
)

# Initialize Gemini model (multimodal)
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# Get the response from the model
response = llm(formatted_prompt)
print(response.content)

/var/folders/9w/3l6rxjcs3bgfl3v7b2f66llr0000gn/T/ipykernel_77878/2457562682.py:44: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(formatted_prompt)


The image features a tabby cat wearing orange sunglasses. The cat has green eyes and a white chest. It's sitting on what appears to be a sofa, with various cushions visible in the background. The image is well-lit and has a shallow depth of field, focusing attention on the cat's face.


In [ ]:
from langchain_core.runnables import RunnableLambda

top_k = 2

def retrieve_images(query):
    return [encode_image("image.jpg") for i in range(top_k)]

def print_thing(thing):
    print(thing)
    return thing

def expand_context_images(message):
    new_message = {}
    new_message["query"] = message["query"]
    for i, image in enumerate(message["context_images"]):
        new_message[f"context{i}"] = image
    
    return new_message

system_message = (
    "You are an assistant for question-answering tasks."
    "Use the images of retrieved context to answer the question."
    "If you don't know the answer, just say that you don't know."
    "Use around three sentences for your answer and keep it concise."
    "Do not ever mention that you are using the context as source."
) 

user_message = [
    {
        "type": "text",
        "text": "{query}",
    }
] + [
    {
        "type": "image_url",
        "image_url": {"url": "data:image/jpeg;base64,{placeholder}".replace("placeholder", f"context{i}")},
    }
    for i in range(top_k)
]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        (
            "user",
            user_message,
        ),
    ]
)
# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_message),
#         (
#             "user",
#             [
#                 {
#                     "type": "text",
#                     "text": "{query}",

#                 },
#                 {
#                     "type": "image_url",
#                     "image_url": {"url": "data:image/jpeg;base64,"},
#                 }
#             ],
#         ),
#     ]
# )

rag_chain = (
    {"context_images": retrieve_images, "query": RunnablePassthrough()}
    | RunnableLambda(expand_context_images)
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What are these images? Are they the same?")

'The images show a cat wearing orange sunglasses. Yes, they are the same.'

In [42]:
prompt

ChatPromptTemplate(input_variables=['context0', 'context1', 'query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.Use the images of retrieved context to answer the question.If you don't know the answer, just say that you don't know.Use around three sentences for your answer and keep it concise.Do not ever mention that you are using the context as source."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=[PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), ImagePromptTemplate(input_variables=['context0'], input_types={}, partial_variables={}, template={'url': 'data:image/jpeg;base64,{context0}'}), ImagePromptTemplate(input_variables=['context1'], input_types={}, partial_variables={}, template={'url': 'data:image/jpeg;base64,{context1}'})], additional_kwargs={}

## Evaluation

In [4]:
# Only run this block for Gemini Developer API
# from google import genai
# genai_client = genai.Client(api_key=GOOGLE_API_KEY)

In [4]:
from preprocessing_utils import load_queries, load_answers

queries = load_queries("papers/selection_queries.txt")
answers = load_answers("papers/selection_answers.txt")

In [5]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase
from deepeval_utils import GenerativeGeminiModel, GeminiJudge

# Replace this with the actual output from your LLM application
actual_output = "I like trains."
generative_llm = GenerativeGeminiModel(model_name="gemini-2.0-flash")
judge_llm = GeminiJudge(model_name="gemini-2.0-flash-lite")

metric = AnswerRelevancyMetric(
    threshold=0.7,
    model=judge_llm,
    include_reason=True
)
test_case = LLMTestCase(
    input="What if these shoes don't fit?",
    actual_output=actual_output
)

# To run metric as a standalone
# metric.measure(test_case)
# print(metric.score, metric.reason)

# result = evaluate(test_cases=[test_case], metrics=[metric])

In [6]:
from deepeval.dataset import EvaluationDataset
from preprocessing_utils import create_dataset


dataset = create_dataset(rag_chain, queries, answers)

Sleeping after 15 queries
Resuming
Sleeping after 15 queries
Resuming
Sleeping after 15 queries
Resuming
Sleeping after 15 queries
Resuming


In [9]:
dataset.save_as(file_type="json", directory="papers/selection_geimin2.0_text", include_test_cases=True)

Evaluation dataset saved at papers/selection_geimin2.0_text\20250406_162552.json!


'papers/selection_geimin2.0_text\\20250406_162552.json'

In [11]:
from deepeval.dataset import EvaluationDataset
dataset = EvaluationDataset()
dataset.add_test_cases_from_json_file(
    file_path="papers/selection_geimin2.0_text/20250406_162552.json",
    input_key_name="input",
    actual_output_key_name="actual_output",
    expected_output_key_name="expected_output",
    context_key_name="context",
    retrieval_context_key_name="retrieval_context",
)

In [12]:
from deepeval.metrics import AnswerRelevancyMetric
from time import sleep

answer_relevancy_metric = AnswerRelevancyMetric(threshold=0.5, model=judge_llm, include_reason=True)

results = []
batch_size = 5
for i in range(0, len(dataset), batch_size):
    partial_dataset = EvaluationDataset(test_cases=dataset.test_cases[i:i+batch_size])
    partial_result = partial_dataset.evaluate([answer_relevancy_metric])
    results.append(partial_result)
    print(f"Sleeping for 70 seconds after processing {i+batch_size} queries")
    sleep(70)
    print("Resuming")

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gemini-2.0-flash-lite, strict=False, 
async_mode=True)...

Evaluating 5 test case(s) in parallel: |██████████|100% (5/5) [Time Taken: 00:11,  2.36s/test case]



Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: gemini-2.0-flash-lite, reason: The score is 1.00 because the output contains a valid JSON object with a 'reason' key, as requested, and it does not contain any irrelevant statements., error: None)

For test case:

  - input: What crystal magnetic structure does EuCuAs have?
  - actual output: EuCuAs has a planar helix magnetic structure with a period of approximately four Eu layers. The magnetic space group of the helical structure with τ = 0.5 is Cc2 (#5.16 in the Opechowski–Guccione (OG) setting). The hexagonal unit cell of EuCuAs is described by the centrosymmetric P63/mmc space group.
  - expected output: EuCuAs is described by the centrosymmetric P63/mmc space group with a hexaognal unit cell. It has an A-type or DP-type antiferromagnetic order or a helical spin arrangement.
  - context: None
  - retrieval context: None


Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, thre

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

Sleeping for 70 seconds after processing 5 queries
Resuming


...

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gemini-2.0-flash-lite, strict=False, 
async_mode=True)...

Evaluating 5 test case(s) in parallel: |██████████|100% (5/5) [Time Taken: 00:14,  2.93s/test case]



Metrics Summary

  - ✅ Answer Relevancy (score: 1.0, threshold: 0.5, strict: False, evaluation model: gemini-2.0-flash-lite, reason: The score is 1.00 because the output correctly provides a JSON object as requested, and there are no irrelevant statements., error: None)

For test case:

  - input: What kind of material is graphdyine and what is BGDY?
  - actual output: Graphdiyne (GDY) is a synthetic carbon allotrope with a layered framework formed by benzoic rings connected by carbon chains. Boron-graphdiyne (BGDY) is a member of the GDY family where single boron atoms replace the hexagonal carbon rings in the GDY lattice, forming a 2D honeycomb lattice with large hexagonal holes. BGDY is formed by boron atoms connected by carbon chains containing diacetylenic linkages.
  - expected output: Graphdiyne (GDY) is a synthetic carbon allotrope with a layered framework formed by benzoic rings connected by carbon chains. Boron-graphdiyne (BGDY) is a member of the GDY family where single bo

✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

Sleeping for 70 seconds after processing 60 queries
Resuming


In [ ]:
from deepeval.metrics import FaithfulnessMetric
from time import sleep

faithfulness_metric = FaithfulnessMetric(
    threshold=0.5,
    model="gpt-4",
    include_reason=True
)

results = []
batch_size = 5
for i in range(0, len(dataset), batch_size):
    partial_dataset = EvaluationDataset(test_cases=dataset.test_cases[i:i+batch_size])
    partial_result = partial_dataset.evaluate([faithfulness_metric])
    results.append(partial_result)
    print(f"Sleeping for 70 seconds after processing {i+batch_size} queries")
    sleep(70)
    print("Resuming")